## Import & Load Model

In [ ]:
import torch
device = torch.device('cuda')

In [ ]:
# project05
import os
import time

import pycuda.autoinit
from utils.yolo_classes import get_cls_dict
from utils.display import open_window, set_display, show_fps
from utils.visualization import BBoxVisualization
from utils.yolo import TRT_YOLO

trt_yolo = TRT_YOLO("yolov4-tiny-416", (416, 416), 4)

In [ ]:
# midterm 
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth'))

## Pre-processing

In [ ]:
# midterm
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [ ]:
# midterm
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

In [ ]:
# midterm
image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

In [ ]:
from jetbot import Robot

robot = Robot()

## Adjust Parameters

In [ ]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.12,description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.04, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.25, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

In [ ]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

## Server Response & Test

In [ ]:
import requests

def openai(image_path):
    url = "http://192.168.99.79:8000/get_openai"    
    with open(image_path, "rb") as image_file:
        files = {"file": image_file}
        response = requests.post(url, files=files)
    
    # Handle the response
    if response.status_code == 200:
        # print("Response content:", response.json())
        print(response.json())
        return response.json()
    else:
        print("Error:", response.status_code, response.text)
        
def openai_text(sign):
    url = "http://192.168.99.79:8000/get_openai_text"    
    response = requests.post(url, data=sign)
    
    # Handle the response
    if response.status_code == 200:
        # print("Response content:", response.json())
        print(response.json())
        return response.json()
    else:
        print("Error:", response.status_code, response.text)

In [ ]:
# img_test = cv2.imread("./detected_img/road_close_2024-12-27 07:00:46.jpeg")
response = openai("./detected_img/road_close.jpeg")
print(response)

## Memory Check

In [ ]:
import psutil

memory = psutil.virtual_memory()
print(f"Total memory: {memory.total / (1024**3):.2f} GB") # 總記憶體
print(f"Available memory: {memory.available / (1024**3):.2f} GB") # 可用記憶體
print(f"Used memory: {memory.used / (1024**3):.2f} GB") # 已使用記憶體

## Yolo Object Detection Test

In [ ]:
pre_t = 0
sign_class={0:"道路封閉", 1:"減速行駛", 2:"加速行駛", 3:"行人(pedestrian)", 4:"鐵路平交道", 5:"停車再開(stop)"}

def yolo_test(change):
    global pre_t
    image = change['new']
    image_resize = cv2.resize(image, (416, 416))
    img = image_resize.copy() 
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
#     print("image: ",image.shape)
#     print("image_resize: ",image_resize.shape)
    start = time.time()
    boxes, confs, clss = trt_yolo.detect(image_resize)
    print(f"class: {sign_class[int(clss[0])]}")
#     print("yolo 耗時：", time.time()-start)
#     print("boxes: ",boxes)
    cv2.imwrite("./detected_img/test_origin.jpeg",image)
    if len(boxes)!=0:
        for i in range(len(clss)):
            cv2.rectangle(img, (boxes[i][0], boxes[i][1]), (boxes[i][2], boxes[i][3]), (0, 0, 255), 1) # wen
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        cv2.imwrite("./detected_img/test.jpeg",img)
        start = time.time()
#         openai("./detected_img/test.jpeg")
        print("openai 耗時：", time.time()-start)

#     print("-- end --")

In [ ]:
yolo_test({'new': camera.value})

## Section of Sign Detection (Thread)

In [ ]:
## global frame Queue
import threading
from queue import Queue
frame_queue = Queue(maxsize=1)
pre_t = 0
speed_factor = 1
steering_factor = 1
sign_class={0:"道路封閉", 1:"減速行駛(速限60)", 2:"加速行駛(速限30)", 3:"行人(pedestrian)", 4:"鐵路平交道", 5:"停車再開(stop)"}
sign_filename={0:"block", 1:"max60", 2:"min30", 3:"pedestrain", 4:"railway", 5:"stop"}
used_sign = []

def yolo():
    global pre_t, speed_factor, steering_factor, used_sign
    ALERT_WIDTH = 40 
    while(1):
        image = frame_queue.get()
        image_resize = cv2.resize(image, (416, 416))
        img = image_resize.copy() 
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        start = time.time()
        boxes, confs, clss = trt_yolo.detect(image_resize)
#         print("yolo 耗時：", time.time()-start)
        cv2.imwrite("./detected_img/test_origin.jpeg",image)
        t = time.time()
        if len(boxes)!=0:
            width = boxes[0][2] - boxes[0][0]
        else:
            width = 0
            
        if len(boxes)!=0 and t > pre_t + 5 and width > ALERT_WIDTH and (int(clss[0])not in used_sign):
            used_sign.append(int(clss[0]))
            print(f"第{t}秒 class: {sign_class[int(clss[0])]}")
            
#             0 道路封閉
#             1 減速行駛
#             2 加速行駛
#             3 行人(pedestrian)
#             4 鐵路平交道
#             5 停車再開(stop)
            
            if int(clss[0]) == 0:
                speed_factor = 0
                steer_factor = 0
            
            elif int(clss[0]) in [2, 3]:
                speed_factor = 0.9
                
            elif int(clss[0]) == 1:
                speed_factor = 1.2
                
            elif int(clss[0]) == 4:
                speed_factor = 0
                steering_factor = 0
                time.sleep(5)
                speed_factor = 1
                steering_factor = 1
                
            elif int(clss[0]) == 5:
                speed_factor = 0
                steering_factor = 0
                time.sleep(2)
                speed_factor = 1
                steering_factor =1

            for i in range(len(clss)):
                cv2.rectangle(img, (boxes[i][0], boxes[i][1]), (boxes[i][2], boxes[i][3]), (0, 0, 255), 1) # wen
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            cv2.imwrite("./detected_img/test.jpeg",img)
            start = time.time()
            pre_t = time.time() # 更新時間
#             print("輸入照片："+"./standard_sign/"+sign_filename[int(clss[0])]+".jpeg")
            openai("./standard_sign/"+sign_filename[int(clss[0])]+".jpeg")
            print("-- openai 耗時：", time.time()-start,"--")
#             pre_t = time.time() # 更新時間
            
        if image is None:
            print("kill thread")
            frame_queue.task_done()
            break
#     print("-- end --")

## Section of Path Planning

In [ ]:
# midterm
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last #midterm
    global robot, count, speed_left, speed_right, stop_ignore, railway_ignore, pedestrian_ignore # project5
    global speed_factor, steering_factor
    ALERT_WIDTH = 40 # project5
    
    image = change['new']

    ######################## midterm #############################
    
    xy = model_trt(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value * speed_factor
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle

    steering_slider.value = (pid + steering_bias_slider.value) * steering_factor

    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)

    
    
    ####################### project5 #############################
#     yolo(image)
    if not frame_queue.full():
#         print("put img")
        frame_queue.put(image)
    

sign_detection = threading.Thread(target=yolo)
sign_detection.start()

pre_t = time.time()
# execute({'new': camera.value})

## Execute Whole Task

In [ ]:
pre_t = time.time()
used_sign = []
speed_factor = steering_factor = 1
camera.observe(execute, names='value')

## Stop Execution & Release Camera

In [ ]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()


In [ ]:
camera.stop()